In [4]:
import pandas as pd

In [5]:
packets = pd.read_csv('vdg1nb.csv')

In [6]:
packets.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,172.217.12.170,192.168.8.219,TLSv1.2,107,Application Data
1,2,0.000108,192.168.8.219,172.217.12.170,TCP,66,55530 > 443 [ACK] Seq=1 Ack=34 Win=2047 Len=...
2,3,0.511880,77.234.46.106,192.168.8.219,HTTP,254,HTTP/1.1 200 OK
3,4,0.512007,192.168.8.219,77.234.46.106,TCP,66,55557 > 80 [ACK] Seq=1 Ack=181 Win=2045 Len=...
4,5,0.517461,192.168.8.219,77.234.46.106,HTTP,320,GET /R/A1MKIDdENTAzMjMwRTMxQTQ5MEJBNUQ2QTcyNTR...


In [7]:
packets.shape

(92349, 7)

In [8]:
packets_p = packets.groupby('Protocol').Source.count()

In [9]:
packets_p

Protocol
ARP           82
BROWSER        2
DNS          363
HTTP          27
ICMP          35
ICMPv6         4
MDNS          18
NBSS           5
OCSP           1
SSLv2         10
TCP        81481
TLSv1.2     2316
TLSv1.3     8005
Name: Source, dtype: int64

In [10]:
packets_p.sort_values()

Protocol
OCSP           1
BROWSER        2
ICMPv6         4
NBSS           5
SSLv2         10
MDNS          18
HTTP          27
ICMP          35
ARP           82
DNS          363
TLSv1.2     2316
TLSv1.3     8005
TCP        81481
Name: Source, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
packets.columns

Index(['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Length', 'Info'], dtype='object')

In [13]:
No = tf.feature_column.numeric_column('No.')
Time = tf.feature_column.numeric_column('Time')
Source = tf.feature_column.numeric_column('Source')
Destination = tf.feature_column.numeric_column('Destination')
Length = tf.feature_column.numeric_column('Length')
Info = tf.feature_column.numeric_column('Info')

In [25]:
protocol = tf.feature_column.categorical_column_with_vocabulary_list('Protocol',['OCSP','BROWSER','ICMPv6','NBSS','SSLv2','MDNS','HTTP','ICMP','ARP','DNS','TLSv1.2','TLSv1.3','TCP'])

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
feat_cols = [No,Time,Source,Destination,Length,Info,protocol,]

In [28]:
x_data = packets.drop('Protocol',axis=1)

In [29]:
x_data.head()

,No.,Time,Source,Destination,Length,Info
0,1,0.000000,172.217.12.170,192.168.8.219,107,Application Data
1,2,0.000108,192.168.8.219,172.217.12.170,66,55530 > 443 [ACK] Seq=1 Ack=34 Win=2047 Len=...
2,3,0.511880,77.234.46.106,192.168.8.219,254,HTTP/1.1 200 OK
3,4,0.512007,192.168.8.219,77.234.46.106,66,55557 > 80 [ACK] Seq=1 Ack=181 Win=2045 Len=...
4,5,0.517461,192.168.8.219,77.234.46.106,320,GET /R/A1MKIDdENTAzMjMwRTMxQTQ5MEJBNUQ2QTcyNTR...


In [30]:
labels = packets['Protocol']

In [31]:
labels.head()

0    TLSv1.2
1        TCP
2       HTTP
3        TCP
4       HTTP
Name: Protocol, dtype: object

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.3,random_state=101)

In [34]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [35]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=13)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/qf/x4zt60d17593p_nh2sl0p2v40000gn/T/tmpkkl4c4ft', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb25a77390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.


ValueError: Feature Protocol is not in features dictionary.